<a href="https://colab.research.google.com/github/fernandoespinosa/3666-applied-natural-language-processing/blob/master/SCS_3666_018_FernandoEspinosa_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



##In this assignment, you will use a predefined list of Positive and negative words which is been published by Minqing Hu and Bing Liu in their paper named "Mining and Summarizing Customer Reviews."

##The word list is available for download from http://www.cs.uic.edu/~liub/FBS/opinion-lexicon-English.rar


##References:
Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." Proceedings of the ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, Washington, USA





In [1]:
#Setup your code
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

##Authorize Google Colab & Read the positive and Negative Sentiment words from a Text File downloaded from  http://www.cs.uic.edu/~liub/FBS/opinion-lexicon-English.rar

You need to download those files from above location and save into your google drive or local drive location frim where you will read the files in your program.

In [2]:
import nltk
from collections import Counter

POSITIVE = 'positive'
NEGATIVE = 'negative'
BOTH = 'both'
UNKNOWN = 'unknown'


from google.colab import drive
drive.mount('/gdrive')

# Change below path to where you have saved your downloaded text files
%cd /gdrive/My Drive/Colab Notebooks/opinion-lexicon-English/


# Build the  positive word lists from files
with open('positive-words.txt', 'rb') as f:
         positive_set = set(nltk.word_tokenize(f.read().decode('ISO-8859-1')))

# Build the  negative word lists from files
with open('negative-words.txt', 'rb') as f:
         negative_set = set(nltk.word_tokenize(f.read().decode('ISO-8859-1')))


# Now we need to remove the headers(which are not part of the word list) from the word list files.
# we can use set difference function to get rid of the header part
positive_words = positive_set.difference(negative_set)
negative_words = negative_set.difference(positive_set)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Colab Notebooks/opinion-lexicon-English


In [3]:
import itertools

print(f'positive_words: {len(positive_words)} — sample: {[w for w in itertools.islice(positive_words, 10)]}')
print(f'negative_words: {len(negative_words)} — sample: {[w for w in itertools.islice(negative_words, 10)]}')


positive_words: 2004 — sample: ['well-intentioned', 'victory', 'righteousness', 'stainless', 'thrilling', 'compactly', 'valiantly', 'precisely', 'rightfully', 'stability']
negative_words: 4784 — sample: ['overwhelmingly', 'overwhelmed', 'unforeseen', 'mockingly', 'overstatement', 'irrational', 'back-logged', 'leaking', 'hassled', 'languor']


## Define the Parse_Sentiment Function

In [4]:
def parse_sentiment(text):
  sentences = [[word.lower() for word in nltk.word_tokenize(sentence)] for sentence in nltk.sent_tokenize(text)]
  sents, words = count_sentiment(sentences)
  print(words)
  total = sum(words.values())


from collections import Counter

def count_sentiment(sentences):
  sents = Counter()
  words = Counter()

  for sentence in sentences:
    sentiment = sentimentalize(sentence)
    sents[sentiment] += 1
    words[sentiment] += len(sentence)

  return sents, words

def sentimentalize(words):
  positive_count = len(positive_words.intersection(words))
  negative_count = len(negative_words.intersection(words))
  if positive_count > 0 and negative_count > 0:
    return BOTH
  elif positive_count > 0:
    return POSITIVE
  elif negative_count > 0:
    return NEGATIVE
  else:
    return UNKNOWN

## Question: Parse the target Article directly from URL and calculate the overall sentiment score as being expressed in the news article

In [5]:
!pip install inscriptis


In [6]:
!curl -IL 'https://www.nytimes.com/2017/01/26/arts/dance/rehearse-ice-feet-repeat-the-life-of-a-new-york-city-ballet-corps-dancer.html' | head -n 10

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  276k    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
HTTP/2 200 
x-b3-traceid: aeccc86240664716b8a868cc4e0535b4
x-nyt-data-last-modified: Fri, 20 Sep 2024 21:44:56 GMT
last-modified: Fri, 20 Sep 2024 21:44:56 GMT
x-scoop-last-modified: 2017-12-22T02:31:06.552Z
x-pagetype: vi-story
x-xss-protection: 1; mode=block
x-content-type-options: nosniff
content-type: text/html; charset=utf-8
x-envoy-upstream-service-time: 190


In [7]:
import urllib.request
from inscriptis import get_text

url = 'https://www.nytimes.com/2017/01/26/arts/dance/rehearse-ice-feet-repeat-the-life-of-a-new-york-city-ballet-corps-dancer.html'
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'})
html = urllib.request.urlopen(req).read().decode('utf-8')
text = get_text(html)
parse_sentiment(text)

Counter({'both': 292, 'unknown': 192, 'positive': 146, 'negative': 95})
